In [25]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
#if gpu is available set device to cuda else cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#block size is size of blocks within the training data. batch_size is how many blocks are done in parallel
block_size = 8 #block size/ sequence length
batch_size = 4 #how many blocks we want at the same time
max_iters = 2000 #how many iterations we want to do
learning_rate = 1e-4 #more learning rates -> 3e-3, 3e-4, 1e-3, 1e-4
eval_iters = 200 #reporting the loss
n_embd = 260 #parameter denoting the length of the vector holding attribute about tokens OR 
             #the total dimensions captured from all the concatenated heads
n_head = 4 # no. of heads we have running 
n_layer = 4 #no. of layers/blocks(decoder blocks)
dropout = 0.2 #dropping 20% of neurons from the neural net

cuda


In [26]:
#Reading the file and visualizing first 200 words
with open('Dracula.txt', 'r', encoding='utf-8') as f:
    text = f.read()
#print(text[:100])
#What characters are in the book and the total number of them are printed
#A Vocabulary set of words and symbols
chars = sorted(set(text))
#print(chars)
vocab_size = len(chars)
#print(vocab_size)

In [27]:
#dictionaries for string to int and vice-versa, ch - character, i - char's index value i.e. equivalent int. value
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
#string 's' is input
encode = lambda s: [string_to_int[c] for c in s] 
#list of integers 'l' is input
#.join to concatenate the encoded integers(i.e. characters) back into a string
decode = lambda l: ''.join([int_to_string[i] for i in l]) 

#enc = encode('dracula')
#print(enc)
#dec = decode(enc)
#print(dec)

#entire text data of the book, encoded
data = torch.tensor(encode(text), dtype = torch.long)
#print(data[:100])

In [28]:
#training set is 80% of total data to decrease complete duplicacy and increase variety
#generate text 'like' the training data and not just have it 'memorized' entirely
n = int(0.8*len(data))
#training set 80%
train_data = data[:n]
#validation set 20%
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

a, b = get_batch('train')
print('inputs:')
print(a)
print('targets:')
print(b)

inputs:
tensor([[57,  1, 73, 61, 54, 73,  1, 73],
        [ 1, 61, 58,  1, 76, 54, 72,  0],
        [73,  1, 76, 62, 65, 65,  1, 58],
        [ 1, 28,  9,  7,  1, 28,  9,  1]], device='cuda:0')
targets:
tensor([[ 1, 73, 61, 54, 73,  1, 73, 61],
        [61, 58,  1, 76, 54, 72,  0, 64],
        [ 1, 76, 62, 65, 65,  1, 58, 54],
        [28,  9,  7,  1, 28,  9,  1, 40]], device='cuda:0')


In [29]:
#x are 'predictions' and y are 'targets' within the block size(blocks within the training data of 80%)
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target  = y[t]
    print("When input is", context, "target is", target) 

When input is tensor([98]) target is tensor(1)
When input is tensor([98,  1]) target is tensor(1)
When input is tensor([98,  1,  1]) target is tensor(1)
When input is tensor([98,  1,  1,  1]) target is tensor(1)
When input is tensor([98,  1,  1,  1,  1]) target is tensor(1)
When input is tensor([98,  1,  1,  1,  1,  1]) target is tensor(1)
When input is tensor([98,  1,  1,  1,  1,  1,  1]) target is tensor(1)
When input is tensor([98,  1,  1,  1,  1,  1,  1,  1]) target is tensor(1)


In [30]:
#decorator to ensure pytorch doesn't use gradients at all, reduces computation and memory usage, better performance.
#gradients used during training, no training done here so...no_grad()
#function for reporting and evaluating how the model is performing
@torch.no_grad()
def estimate_loss():
    out = {}
    #model set to evaluation mode, dropout disabled 
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    #model set to training mode, dropout enabled 
    model.train()
    return out

In [31]:
class Head(nn.Module):
    #one head of self-attention

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        #.register_buffer registers the nolookahead masking to the model state,
        #no need to reinitialize the heads for every forward and backward pass, saving computation and training time
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        #Input of size (batch, time-step, channels), Output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x) #(B,T,hs) hs->headsize
        q = self.query(x) #(B, T, hs)
        #compute attention scores("affinities")
        #.transpose sets up a timestable format, allowing us to multiply and flipping -2dim with -1dim(last dim)
        #k.shape[-1]**-0.5 is the scaling done to enable us to distinct and scaled down head size dim 
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T) Dot Producting
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) masked_fill to prevent lookahead
        #all 0 converted to '-inf' for softmax to exponent
        #Making model more confident in its predictions by assigning and highlighting appropriate attention
        #when value/attention score is big we want the model to focus more on that and concurrently learn more from it.
        wei = F.softmax(wei, dim=-1) #(B, T, T) 
        wei = self.dropout(wei)
        #perform the weighted aggregation of the values, i.e. adding Values to Keys and Queries final Output 
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


class MultiHeadAttention(nn.Module):
    #multiple heads of self-attention in parallel

    def __init__(self, num_heads, head_size):
        super().__init__()
        #ModuleList -> Holds submodules in a list, essentially having "Head"s in parallel for each head(num_heads)
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        #.proj ->  transforms the concatenated output from a size of (head_size * num_heads) to a size of n_embd.
        #purpose is to combine information from all the heads and transform it into a form that can be used by the subsequent layers or models
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        #concatenating each head together along the last dimension(i.e. the C/Features, last dim)
        out = torch.cat([h(x) for h in self.heads], dim=-1) #dim -1 meaning in C/Features of (B, T, C/Features)
        out = self.dropout(self.proj(out))
        return out

        
class FeedForward(nn.Module):
    #a simple linear layer followed by a non-linearity

    def __init__(self, n_embd):
        super().__init__()
        #Sequential “chains” outputs to inputs sequentially for each subsequent module
        #Output of .Linears/output shape will be of dimension (n_embd by n_embd). Also making sure inner dimensions line up(i.e. 4*n_embd)
        #ReLU -> introduces non-linearity by outputting the input directly if it's positive, and zero otherwise
        #dropout ->  randomly drop units (along with their connections) from the neural network during training, prevents overfitting.
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
#Transformer block: communication followed by computation

    def __init__(self, n_embd, n_head):
        super().__init__()
        #n_embd: embedding dimension, n_head: the no. of heads we'd like
        #head_size: no. of features each head will be capturing in MultiHeadAttention
        head_size = n_embd // n_head
        #self-attention
        self.sa = MultiHeadAttention(n_head, head_size)
        #feed-forward
        self.ffwd = FeedForward(n_embd)
        #2 layer normalizations
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        #DECODER architecture -> selfattention, add a norm, feedforward, add a norm
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

#logits are bunch of floating point numbers that are normalized
#they are probability distributions of what we want to predict
#normalization is the division of single element with sum of all elements
# B is Batch, T is Time dim.(value we don't know), C is Channels/how many are there or the vocab size.
class GPTLanguageModel(nn.Module): #GPT - Generative Pre-trained Transformer
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #.blocks is how many blocks/layers we have running sequentially
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm, added at the end of decoders
        self.lm_head = nn.Linear(n_embd, vocab_size) #final linear transformation nn.Linear, to work with softmax

        #making sure weights are initialized properly, if standard deviation is too high, outliers occur and 
        #learning is difficult, if it's low, no learning is done, no oppurtunity for varied parameters to learn
        self.apply(self._init_weights)

    #initializing weights around certian standard deviations
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        #logits = self.token_embedding_table(index)
        B, T = index.shape #unpacking B and T
        
        #index and targets are both (B,T) tensor of integers
        #Broadcasting Semantics - Rules about operations done on Tensors
        tok_emb = self.token_embedding_table(index) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)  
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x) #(B,T,C) #finallayernorm
        logits = self.lm_head(x) #(B,T,vocab_size) #nn.linearTransformation before softmax

        if targets is None:
            loss = None
        else:
            #blending B and T since C is what is being observed and
            #as long as logits and targets have same B and T, should be fine
            #since doc says parameter to be in the form (N, C) where N is our Batch(No. of batches)
            #i.e. in the form -> (B*T(made into 1), C) instead of B, T, C
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss    

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            # focusing only on the last time step
            logits = logits[:, -1, :] #(B, C)
            #applying softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B, C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append sample index to the running sequence 
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)
#torch.long same as int64

#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [32]:
#Pytorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step:{iter}, train loss:{losses['train']:.3f}, val loss:{losses['val']:.3f}")
    #sampling a batch of data
    #xb-inputs, yb-targets
    #evaluating the loss
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    #.zero_grad usually used in RNN
    optimizer.zero_grad(set_to_none=True)#set to none instead of zero gradient accumulated by Pytorch
    loss.backward()
    optimizer.step()
print(loss.item())

step:0, train loss:4.616, val loss:4.613
step:200, train loss:2.703, val loss:2.714
step:400, train loss:2.558, val loss:2.545
step:600, train loss:2.467, val loss:2.478
step:800, train loss:2.421, val loss:2.426
step:1000, train loss:2.352, val loss:2.369
step:1200, train loss:2.358, val loss:2.346
step:1400, train loss:2.333, val loss:2.353
step:1600, train loss:2.276, val loss:2.287
step:1800, train loss:2.236, val loss:2.293
1.8544710874557495
